# Reconocimiento de fonemas usando CTC

In [1]:
import json
import torch
import torch.utils.data as data
import torchaudio
from matplotlib import pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Implementación del dataset y el dataloader

### Dataset
Me convierte los datos crudos para que puedan ser usados por `Dataloader`. Me permite implementar una fucnión `__getitem__()` en la cual leemos los datos y devolvemos por ejemplo el wav y la transcripción de cada dato.

In [16]:
train_json = 'data/train.json'
test_json = 'data/test.json'
valid_json = 'data/dev.json'

class TimitDataset(data.Dataset):
    def __init__(self, json_file):
        with open(json_file, 'r') as f:
            self.datos_json = json.load(f)

    def __len__(self):
        return len(self.datos_json)
    
    def __getitem__(self,key):
        wavdir = self.datos_json[key]['wav']
        duration = self.datos_json[key]['duration']
        phn = self.datos_json[key]['phn']
        return wavdir, duration, phn
    
train_dataset = TimitDataset(train_json)
test_dataset = TimitDataset(test_json)
valid_dataset = TimitDataset(valid_json)

# Ejemplo de uso de Dataset           
wavdir, dur, phn = valid_dataset['mrws1_sx230']
        
        

In [17]:
phn

'sil ah l aw l iy w ey hh iy er sil b ah r ae sh sil n l ay z aa l eh r er z sil'

### DataLoader
La función principal es agrupar en batches los items que provienen del dataset y, si es necesario, padear los items para que el batch sea de igual ancho.